In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:

def create_vectorstore(flag):
    if flag == False:
        flag=True
        loader = Docx2txtLoader(r"data/Khulna University of Engineering.docx")
        documents=loader.load()
        # text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30,separator="\n")
        # docs=text_splitter.split_documents(documents=documents)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        vectorstore=FAISS.from_documents(docs,hf)
        vectorstore.save_local('vectorstore')
        return vectorstore
    else:
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        new_vectorstore=FAISS.load_local("vectorstore",hf)
        return new_vectorstore

In [ ]:
from langchain import HuggingFacePipeline
path=r'KUETLLM/KUETLLM_zephyr7b_beta'
tokenizer = AutoTokenizer.from_pretrained(path,
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained(path,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
    
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})


In [ ]:
retriever=create_vectorstore(False).as_retriever()

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# torch.cuda.empty_cache()

In [ ]:
def ans_ret(inp,history):
    ans=rag_chain.invoke(inp)
    k=ans.split("Based on the text material")
    if len(k)==2:
        return k[0]
    return ans

In [ ]:
import gradio as gr

demo = gr.ChatInterface(fn=ans_ret, examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()